# Math 134C Week 10

In [1]:
import math
import numpy as np
from scipy.stats import norm
from scipy.stats.mstats import gmean

def black_scholes_call(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    d2 = d1 - sigma * np.sqrt(dt)
    call_price = S_t * np.exp(-delta * dt) * norm.cdf(d1) - K * np.exp(-r * dt) * norm.cdf(d2)
    return call_price

def black_scholes_put(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    d2 = d1 - sigma * np.sqrt(dt)
    call_price = - S_t * np.exp(-delta * dt) * norm.cdf(-d1) + K * np.exp(-r * dt) * norm.cdf(-d2)
    return call_price

def call_Delta(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))  
    return np.exp(-delta * dt) * norm.cdf(d1)

def put_Delta(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))  
    return -np.exp(-delta * dt) * norm.cdf(-d1)

def Gamma(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    return np.exp(-delta * dt) * norm.pdf(d1)/(S_t * sigma * np.sqrt(dt)) 

def call_Theta(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    d2 = d1 - sigma * np.sqrt(dt)
    return delta * S_t * np.exp(-delta * dt)*norm.cdf(d1) - r * K * np.exp(-r*dt)*norm.cdf(d2)-(K * np.exp(-r*dt) * sigma * norm.pdf(d2))/ (2 * np.sqrt(dt)) 

def put_Theta(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    d2 = d1 - sigma * np.sqrt(dt)
    return call_Theta(S_t, K, dt, r, sigma, delta) + r * K * np.exp(-r*dt) - delta * S_t * np.exp(-delta * dt)

def put_given_call(C_t, S_t, K, r, dt):
    return C_t - S_t + K * math.exp(-r * dt)
def call_given_put(P_t, S_t, K, r, dt):
    return P_t + S_t - K * math.exp(-r * dt)

def one_step_binomial(r, dt, u, d, Vu, Vd, delta = 0):
    q = (math.exp((r - delta) * dt) - d)/(u-d)
    if not (1 > q > 0): 
        print("Arbitrage opportunity exists")
        return -1
    return math.exp(-r * dt)*(q*Vu+(1-q)*Vd)    

def stock_price_simulation(T, N, r, delta, sigma, S0):
    dt = float(T) / N
    t = np.linspace(0, T, N+1)
    S = np.zeros(N+1)
    S[0] = S0
    for i in range(1, N+1):
        Z = np.random.normal(0, 1)
        S[i] = S[i-1] * np.exp((r - delta - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    return t, S

**Quiz 9.1**
![](./img/week10_1.png)

In [7]:
q = (np.exp(0.05-0.04) - 0.6)/(1.3-0.6)
print(q)
Cu = np.exp(-0.05) * (q * 59)
print(Cu)
P = np.exp(-0.05) * (q * (Cu-5))
print(P)

0.5857859529773828
32.87579326094541
15.532861292062652


**Quiz 9.2**
For this problem please assume the risk-free annual interest rate is 5.35\% and the underlying stock has a current price of 41.75 and volatility of 0.34. Also assume there are 365 days until expiration.

Using the Monte Carlo methodology with one day time steps, compute the value of an exotic Asian-style geometric average price European put option with a strike price of 55 using 10,000 trials.

Please round your answer to the nearest integer.

In [8]:
T = 1
N = 365
r = 0.0535
delta = 0
sigma = 0.34
S0 = 41.75
K = 55
n_trials = 10000

price_sum = 0
for i in range(n_trials):
    t,S = stock_price_simulation(T, N, r, delta, sigma, S0)
    Smean = gmean(S)
    price_sum += np.exp(-r) * max(0, K - Smean)

print(price_sum/n_trials)

12.260054536502018


**Quiz 9.3**
For this problem please assume the risk-free annual interest rate is 5.35% and the underlying stock has a current price of 41.75 and volatility of 0.34. Also assume there are 365 days until expiration.

Using the Monte Carlo methodology, compute the value of an exotic Asian-style arithmetic average strike European put option using 10,000 trials.

Please round your answer to the nearest integer.

In [10]:
T = 1
N = 365
r = 0.0535
delta = 0
sigma = 0.34
S0 = 41.75
K = 55
n_trials = 10000

price_sum = 0
for i in range(n_trials):
    t,S = stock_price_simulation(T, N, r, delta, sigma, S0)
    Smean = S.mean()
    price_sum += np.exp(-r) * max(0, Smean-S[-1])

print(price_sum/n_trials)

2.7140369115642207


**Quiz 9.4**
In this problem you must  construct a 3-period binomial tree for modeling the price movements of a stock.  Assume the following form for the up and down movements in the stock price over each period h:

$$u = e^{(r-\delta)h+\sigma\sqrt{h}}$$
$$d = e^{(r-\delta)h-\sigma\sqrt{h}}$$

Also you are given:

![](./img/week10_2.png)

To be clear, the compound option expires 3 months from today, while the underlying 6-month 100-strike European put option starts 3 months from today and expires 9 months from today.

In [17]:
r = 0.1
delta = 0.08
h = 0.25
sigma = 0.32
S0 = 110

u = np.exp((r-delta)*h + sigma * np.sqrt(h))
d = np.exp((r-delta)*h - sigma * np.sqrt(h))
print(u,d)
Su = S0 * u
Sd = S0 * d
Suu = S0 *u * u
Sud = S0* u *d 
Sdd = S0 * d*d
Suuu = S0 * u * u * u
Suud = S0 * u * u *d
Sudd = S0 * u * d * d
Sddd = S0 * d * d * d
print(Su, Sd, Suu, Sud, Sdd, Suuu, Suud, Sudd, Sddd)

q = (np.exp((r-delta)*h)-d)/(u-d)
print(q)

Pu = np.exp(-r * 2 * h)*(q**2 * 0 + 2 * q * (1-q) * 0 + (1-q)**2 * (100 - Sudd))
Pd = np.exp(-r * 2 * h)*(q**2 * 0 + 2 * q * (1-q) * (100-Sudd) + (1-q)**2 * (100 - Sddd))

print(Pu, Pd)

P = np.exp(-r * h) * (q * (5-Pu))
print(P)

1.1793931187113906 0.8564151774836135
129.73324305825298 94.20566952319749 153.00649413101584 111.1055183792585 80.67916518467182 180.45480629627485 131.0370838273594 95.15245224218155 69.09486157086049
0.460085115444434
1.3441816833864326 10.860590913483586
1.640459170623296


**Quiz 9.5**
The initial price of a stock is 45. The stock does not pay dividends.

The continuously compounded risk-free annual interest rate is 10%.

An American call option on the stock expires in 1 year and has a strike price of 40.

A 3-month European call on a put with exercise price 0.80 allows the owner to buy a European put option expiring at the end of year 1 on the same stock with strike price 40.

After 3 months, on the day the compound option expires, the price of the stock is 48, and the American call option has a price of 11.82, with 9 months left to expiration.

Calculate the payoff of the call on put compound option on the day it expires.

In [19]:
S0 = 45
delta = 0
T = 1
K = 40


P = put_given_call(11.82, 48, 40, 0.1, 0.75)
print(P)
print("The payoff of this call-on-put option is", P - 0.8)

0.9297394531421119
The payoff of this call-on-put option is 0.12973945314211188


**Quiz 9.6**
![](./img/week10_3.png)

**Quiz 9.7**
![](./img/week10_4.png)

statements II, III are true

**Quiz 9.8**
![](./img/week10_5.png)
True or False:  Pin risk for a European call option is the risk that the call option's delta flips between zero and one as the option flips from being slightly out of the money to slightly in the money, making delta hedging difficult.